In [1]:
require 'image'
require 'nn'
require 'torch'
require 'optim'
torch.setdefaulttensortype('torch.FloatTensor')
net = torch.load('./single-layer(52x52)-for minibatch.t7')
print(net:type())
trainset = torch.load('trainset.t7')
print(trainset.data:type())

torch.FloatTensor	


torch.FloatTensor	


In [2]:
Tsize = trainset.data:size(1)
dat = torch.FloatTensor(Tsize,3,52,52)
lab = torch.FloatTensor(Tsize)
for i=1,Tsize do
    dat[i] = image.scale(trainset.data[i],52,52)
    lab[i] = trainset.class[i]
end

In [3]:
mean = {}
stdv  = {} 
for i=1,3 do
    mean[i] = trainset.data[{ {}, {i}, {}, {}  }]:mean() -- mean estimation
    print('Channel ' .. i .. ', Mean: ' .. mean[i])
    trainset.data[{ {}, {i}, {}, {}  }]:add(-mean[i]) -- mean subtraction

    stdv[i] = trainset.data[{ {}, {i}, {}, {}  }]:std() -- std estimation
    print('Channel ' .. i .. ', Standard Deviation: ' .. stdv[i])
    trainset.data[{ {}, {i}, {}, {}  }]:div(stdv[i]) -- std scaling
end

Channel 1, Mean: 0.32569848013038	


Channel 1, Standard Deviation: 0.20863035193909	


Channel 2, Mean: 0.2611643670599	


Channel 2, Standard Deviation: 0.16150318896284	


Channel 3, Mean: 0.27235293889338	


Channel 3, Standard Deviation: 0.18717469058089	


In [4]:
Tsize = trainset.data:size(1)
print(Tsize)
shuffle = torch.randperm(Tsize)
Sdata = torch.FloatTensor(Tsize,3,52,52)
Slab = torch.FloatTensor(Tsize)

for i=1, Tsize do
    Sdata[i]:copy(dat[shuffle[i]])
    Slab[i] = lab[shuffle[i]]
end

400	


In [6]:
logger = optim.Logger('losses-nag-single-layer.log')
logger:setNames{'Epoch loss'}
Bsize = 16
Tsize = trainset.data:size(1)
Nbatch = math.floor(Tsize/Bsize)
if(not(Tsize/Bsize - Nbatch == 0)) then
    Nbatch = Nbatch+1 
end
print(Nbatch)



criterion = nn.ClassNLLCriterion()
params, gradParams = net:getParameters()
local optimState = {learningRate = .01, learningRateDecay = 1e-7}
local nEpochs = 50
for epoch = 1, nEpochs do
    Tloss = 0
    for i =1,Nbatch do
        if(i == Nbatch) then
            a = (i-1)*Bsize+1;b = Tsize;
        else
            a = (i-1)*Bsize+1;b = i*Bsize; end

        local inp = torch.FloatTensor(b-a+1,3,52,52)
        local lab = torch.FloatTensor(b-a+1)

        inp:copy(Sdata[{{a,b},{},{},{}}])
        lab:copy(Slab[{{a,b}}])

       function feval(params)
          gradParams:zero()

          local outputs = net:forward(inp)
          local loss = criterion:forward(outputs, lab)
          local dloss_doutputs = criterion:backward(outputs, lab)
          net:backward(inp, dloss_doutputs)
          Bloss = loss
--           print('epoch number '..epoch.. ' batch number is ' .. i ..' loss is '..loss)
          return loss, gradParams
       end

       optim.nag(feval, params, optimState)
        Tloss = Tloss + Bloss        
    end
    Tloss = Tloss/Nbatch
    logger:add{Tloss}
    print('epoch number '..epoch..' loss is '..Tloss)
    
end

logger:style{'+-'}
logger:plot()

25	


epoch number 1 loss is -0.020025755614042	


epoch number 2 loss is -0.02002782009542	


epoch number 3 loss is -0.020030032098293	


epoch number 4 loss is -0.020032393857837	


...

epoch number 46 loss is -0.02028536438942	


epoch number 47 loss is -0.020307534486055	


epoch number 48 loss is -0.020332786515355	


epoch number 49 loss is -0.020361877381802	


epoch number 50 loss is -0.020395866408944	
